# Copy Number Pipeline

In [1]:
from __future__ import print_function
import os.path
import numpy as np
import pandas as pd
from IPython.display import Image,display
import dalmatian as dm
from IPython.core.display import HTML 
from src.CCLE_postp_function import *
import sys
sys.path.insert(0, '../JKBio/')
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
import rpy2
from taigapy import TaigaClient
tc = TaigaClient()
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [2]:
workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"
refworkspace="broad-firecloud-ccle/DepMap_WES_CN_hg38"
source1="ibm"
source2="ccle"
source3="ccle"
sample_set_id = "19Q4"
release = sample_set_id
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"

In [3]:
%%R
genome_version <- 'hg38'
release <- '19Q4'
hg38_cyto_band_reference <- '../JKBio/data/hg38_cytoband.gz'
new_samples_copy_number_broad_wes <- 'temp/cnv_ccle.called.seg'
prevfile <- 'wes.19Q3.segmented'

In [4]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

refwm = dm.WorkspaceManager(refworkspace)

In [10]:
newsample = createDatasetWithNewCellLines(refwm, sample_set_id, wmfroms = [wm1, wm2, wm3], sources=[source1, source2, source3],gsfolderto='gs://ccle_bams/wes/',match='ACH', participantslicepos=10, accept_unknowntypes=True)

/Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py:78: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  (samples[extract['id']].isin(forcekeep))]


> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(92)createDatasetWithNewCellLines()
-> if gsfolderto is not None:
(Pdb) n
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(93)createDatasetWithNewCellLines()
-> for i, val in sampless.iterrows():
(Pdb) n
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(94)createDatasetWithNewCellLines()
-> res = os.system('gsutil cp ' + val[extract['bam']] + ' ' + val[extract['bai']] + ' ' + gsfolderto)
(Pdb) n
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(95)createDatasetWithNewCellLines()
-> if res == 256:
(Pdb) res
0
(Pdb) sampless
                              sample_id       LC-SET        PDO aligned_reads  \
82          RP-1561_921_v1_Exome_OnPrem  LCSET-16758  PDO-19525             0   
106       RP-1561_HOTHC_v1_Exome_OnPrem  LCSET-16758  PDO-19525             0   
13

(Pdb) unt
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(102)createDatasetWithNewCellLines()
-> name = val[extract['source']] + '_' + val[extract['id']][:participantslicepos]
(Pdb) 
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(103)createDatasetWithNewCellLines()
-> refsamples = refsamples.append(pd.Series(
(Pdb) 
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(105)createDatasetWithNewCellLines()
-> "CCLE_name": val[extract['name']],
(Pdb) 
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(106)createDatasetWithNewCellLines()
-> "WES_bai": val[extract['bai']],
(Pdb) 
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src/CCLE_postp_function.py(107)createDatasetWithNewCellLines()
-> "WES_bam": val[extract['bam']],
(Pdb) 
> /Users/jeremie/Documents/Projects/BroadInstitute/ccle_processing/src

(Pdb) refsamples.loc[name]
CCLE_name                                                         WM4235
Source                                                              ccle
WES_bai                                    gs://ccle_bams/wes/WM4235.bai
WES_bam                                    gs://ccle_bams/wes/WM4235.bam
allele_fraction_legacy_segments_normal                               NaN
allele_fraction_legacy_segments_tumor                                NaN
allele_fraction_parameters_begin_normal                              NaN
allele_fraction_parameters_begin_tumor                               NaN
allele_fraction_parameters_normal                                    NaN
allele_fraction_parameters_tumor                                     NaN
allelic_counts_entity_id_normal                                      NaN
allelic_counts_entity_id_tumor                                       NaN
allelic_counts_normal                                                NaN
allelic_counts_tumor    

# ReaLign in HG38

In [5]:
bamtoubam= "BamToUnmappedRGBams_MC"
ubamtofilelist = "Generate_uBAM_File_List"
realign="Realign_WES_GATK4"

In [15]:
subid = refwm.create_submission(bamtoubam,sample_set_id,"sample_set","this.samples")
terra.waitForSubmission(refwm, subid)

Successfully created submission 28d7be6b-c9dc-42d2-a4a8-e8c74e2dda3d.


ConnectionError: ('Connection aborted.', OSError("(60, 'ETIMEDOUT')",))

In [ ]:
subid = refwm.create_submission(ubamtofilelist,sample_set_id,"sample_set","this.samples")
terra.waitForSubmission(refwm, subid)

Successfully created submission 8badb033-63f6-419f-8056-b9331f8bc64c.


In [ ]:
subid = refwm.create_submission(realign,sample_set_id,"sample_set","this.samples")
terra.waitForSubmission(refwm, subid)

In [ ]:
submission_id = refwm.create_submission("CNV_sample_XX",etype='sample_set',entity=sample_set_id,expression='this.samples')
terra.waitForSubmission(refwm,submission_id)

In [ ]:
submission_id = refwm.create_submission("Aggregate_CN_seg_files",entity=sample_set_id)
terra.waitForSubmission(refwm,submission_id)

In [ ]:
aggregated = refwm.get_entities('sample_set').loc[sample_set_id]["combined_seg_file"]
aggregated

## deleting unmapped bams

In [ ]:
readgroup_ubams

In [ ]:
hg38_analysis_ready_bam
hg38_analysis_ready_bam_index
hg38_analysis_ready_bam_md5

In [103]:
! gsutil cp $aggregated "temp/cnv_ccle.called.seg"

Copying gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/9b51ecda-b8d3-42bd-a56b-015b94533ee1/aggregate_CN_segments_wrkflw/a565c462-e783-47a4-85c5-a6eba786f03b/call-aggregate_CN_segments/19Q4.called.seg...
- [1 files][110.8 KiB/110.8 KiB]      0.0 B/s                                   
Operation completed over 1 objects/110.8 KiB.                                    


In [108]:
%%R
source('../gkugener/RScripts/load_libraries_and_annotations.R')

2019-10-08 11:41:07::WARNING  R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

2019-10-08 11:41:07::WARNING  R[write to console]: ✔ ggplot2 3.2.0     ✔ tidyr   0.8.3
✔ tibble  2.1.3     ✔ purrr   0.3.2
✔ ggplot2 3.2.0     ✔ forcats 0.4.0

2019-10-08 11:41:07::WARNING  R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()    masks plyr::arrange()
✖ dplyr::collapse()   masks IRanges::collapse()
✖ dplyr::combine()    masks Biobase::combine(), BiocGenerics::combine()
✖ purrr::compact()    masks plyr::compact()
✖ dplyr::count()      masks plyr::count()
✖ dplyr::desc()       masks plyr::desc(), IRanges::desc()
✖ tidyr::expand()     masks S4Vectors::expand()
✖ dplyr::failwith()   masks plyr::failwith()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::first()      masks S4Vectors::first()
✖ dplyr::id()         masks plyr::id()
✖ dplyr::lag()        masks stats::lag()
✖ d

2019-10-08 11:41:08::WARNING  R[write to console]: Arimo for Powerline already registered with pdfFonts().

2019-10-08 11:41:08::WARNING  R[write to console]: Bodoni Ornaments already registered with pdfFonts().

2019-10-08 11:41:08::WARNING  R[write to console]: Bodoni 72 Smallcaps already registered with pdfFonts().

2019-10-08 11:41:08::WARNING  R[write to console]: No regular (non-bold, non-italic) version of . Skipping setup for this font.

2019-10-08 11:41:08::WARNING  R[write to console]: No regular (non-bold, non-italic) version of Brush Script MT. Skipping setup for this font.

2019-10-08 11:41:08::WARNING  R[write to console]: Comic Sans MS already registered with pdfFonts().

2019-10-08 11:41:08::WARNING  R[write to console]: Courier New already registered with pdfFonts().

2019-10-08 11:41:08::WARNING  R[write to console]: Cousine for Powerline already registered with pdfFonts().

2019-10-08 11:41:08::WARNING  R[write to console]: IBM 3270 already registered with pdfFonts()

2019-10-08 11:41:09::WARNING  R[write to console]: The following object is masked from ‘package:XML’:

    addNode


2019-10-08 11:41:09::WARNING  R[write to console]: The following object is masked from ‘package:stringr’:

    boundary


2019-10-08 11:41:09::WARNING  R[write to console]: The following object is masked from ‘package:plyr’:

    join


2019-10-08 11:41:10::WARNING  R[write to console]: Loading required package: RCurl

2019-10-08 11:41:10::WARNING  R[write to console]: Loading required package: bitops

2019-10-08 11:41:10::WARNING  R[write to console]: 
Attaching package: ‘RCurl’


2019-10-08 11:41:10::WARNING  R[write to console]: The following object is masked from ‘package:tidyr’:

    complete


2019-10-08 11:41:10::WARNING  R[write to console]: Loading required package: httr

2019-10-08 11:41:10::WARNING  R[write to console]: 
Attaching package: ‘httr’


2019-10-08 11:41:10::WARNING  R[write to console]: The following object is masked from ‘package:plotly’:

    con

In [178]:
%%R
source("src/CCLE_postp_function.R")

In [109]:
%%R
# Previous release copy number profiles. This line will need to be updated as well
wes.priority.cn.seg.profiles <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.file=prevfile) %>%
  dplyr::select(DepMap_ID, Chromosome, Start, End, Num_Probes, Segment_Mean, Source)

Fetching https://cds.team/taiga/api/datafile?format=metadata&dataset_permaname=segmented-cn-wes-prioritzed-7fe1&datafile_name=wes.19Q3.segmented 
Status 200 


2019-10-08 11:41:29::WARNING  R[write to console]: No encoding supplied: defaulting to UTF-8.



Could not find segmented-cn-wes-prioritzed-7fe1 in cache, requesting from taiga...
Fetching https://cds.team/taiga/api/datafile?format=rds&dataset_version_id=1113ef8f975b4d8c9137fdb09e405448&datafile_name=wes.19Q3.segmented 
Status 200 


2019-10-08 11:41:29::WARNING  R[write to console]: No encoding supplied: defaulting to UTF-8.

2019-10-08 11:41:29::WARNING  R[write to console]: Downloading segmented-cn-wes-prioritzed-7fe1/v33/wes.19Q3.segmented ...



  |======================================================================| 100%
Saving segmented-cn-wes-prioritzed-7fe1 in cache ( 1113ef8f975b4d8c9137fdb09e405448 wes.19Q3.segmented )...


2019-10-08 11:41:34::WARNING  R[write to console]: Saved to cache as 1113ef8f975b4d8c9137fdb09e405448_wes-19q3-segmented.rds



writing /Users/jeremie/.taiga/1113ef8f975b4d8c9137fdb09e405448_wes-19q3-segmented.idx /Users/jeremie/.taiga/segmented-cn-wes-prioritzed-7fe1_wes-19q3-segmented_33.idx 


## If want to reprocess something

In [ ]:
#%%R
#segments_unfiltered <- readr::read_csv('temp/wes.19Q3.segmented.cn')

## post Procesing

### getting blacklist

In [130]:
gsheets = sheets.get(sheeturl).sheets[6].to_frame()

2019-10-08 12:03:51::INFO  URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4?alt=json
2019-10-08 12:03:52::INFO  URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4/values:batchGet?majorDimension=ROWS&valueRenderOption=UNFORMATTED_VALUE&dateTimeRenderOption=FORMATTED_STRING&ranges=Guide&ranges=RNAseq+Embargo&ranges=19Q4&ranges=WES+Embargo&ranges=Blacklist&ranges=Allie%27s+list+-+public+in+19Q2&ranges=full_list&alt=json


In [174]:
rna_ibm_embargo = [i for i in gsheets['RNAseq_IBM_embargo'].values.tolist() if i is not np.nan]
rna_dmc_embargo = [i for i in gsheets['RNAseq_DMC_embargo'].values.tolist() if i is not np.nan]
wes_dmc_embargo = [i for i in gsheets['WES_DMC_embargo'].values.tolist() if i is not np.nan]
blacklist = [i for i in gsheets['blacklist'].values.tolist() if i is not np.nan]

In [168]:
%%R
segments <- processSegments(new_samples_copy_number_broad_wes)
segments <- filterForCCLE(segments)
segments <- interpolateGapsInSegmented(segments)$segs
segments <- extendEndsOfSegments(segments,hg38_cyto_band_reference)
print(segments)
print(head(wes.priority.cn.seg.profiles))
segments_unfiltered <- reprioritizeData(segments, wes.priority.cn.seg.profiles)
# Fill in the gaps on the entire dataset
# Extend start sites to 1, end sites to the end of the chromosome?

[1] 0


2019-10-08 12:18:27::WARNING  R[write to console]: Parsed with column specification:
cols(
  X1 = col_character(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_character(),
  X5 = col_character()
)



# A tibble: 2,094 x 7
   DepMap_ID       seqnames     start       end Num_Probes Segment_Mean Source  
   <chr>           <fct>        <dbl>     <dbl>      <dbl>        <dbl> <chr>   
 1 ccle_ACH-002510 chr1             1  21518734       2734     9.76e- 1 Broad W…
 2 ccle_ACH-002510 chr1      21518735 101921860       6429     9.26e- 1 Broad W…
 3 ccle_ACH-002510 chr1     101921861 132724462       1386     6.34e- 1 Broad W…
 4 ccle_ACH-002510 chr1     132724463 148537132        250     1.12e+ 0 Broad W…
 5 ccle_ACH-002510 chr1     148537133 148549687          1     7.40e-10 Broad W…
 6 ccle_ACH-002510 chr1     148549688 161569704       2475     1.06e+ 0 Broad W…
 7 ccle_ACH-002510 chr1     161569705 161651296          9     1.49e- 7 Broad W…
 8 ccle_ACH-002510 chr1     161651297 216391800       3824     1.07e+ 0 Broad W…
 9 ccle_ACH-002510 chr1     216391801 222131173        199     1.87e+ 0 Broad W…
10 ccle_ACH-002510 chr1     222131174 232963176        962     1.04e+ 0 Broad W…
# … wi

In [180]:
%%R -i wes_dmc_embargo,blacklist

segments_blacklisted <- filterBlackListedLine(blacklist,segments_unfiltered)

# TODO: IF seqnames (CHR) are 1-9 values, append "chr" in front of each! (bulk)
# ifelse(grepl("[0-9]+", new_copy_number$ChrChromosome), "chr"+, "no")
segments_embargo <- filterBlackListedLine(wes_dmc_embargo,segments_unfiltered)

In [183]:
%%R
write.table(segments_unfiltered, file = paste0("temp/wes.",release,".segmented.cn"), sep = ',', quote = F, row.names = F) 

In [184]:
%%R
entrezgenes <- generateEntrezGenes()

In [185]:
%%R
head(corner(entrezgenes))

   EGID SYMBOL CHR    CHRLOC CHRLOCEND
1     1   A1BG  19  58346805  58353499
2    10   NAT2   8  18391281  18401215
3   100    ADA  20  44619518  44651758
4  1000   CDH2  18  27950962  28177130
5 10000   AKT3   1 243488232 243843584


In [186]:
%%R
res <- generateGeneLevelMatrixFromSegments(entrezgenes, segments_unfiltered)

In [187]:
%%R
genematrix_unfiltered <- res$gene_level_data_hg38
corner(genematrix_unfiltered)

           A1BG (1) NAT2 (10) ADA (100) CDH2 (1000) AKT3 (10000)
ACH-000001 1.265172 1.1415254 1.2958359   0.6627515     1.012791
ACH-000002 1.013633 0.9741737 1.0175121   1.4893275     1.012618
ACH-000003 1.031898 1.0958118 1.8768326   0.5364814     1.002498
ACH-000004 1.349290 1.0865089 0.7855092   1.0770610     1.086057
ACH-000005 1.222033 1.1622070 0.7612536   1.0724579     1.120937


In [188]:
%%R
write.table(genematrix_unfiltered, file = paste0('temp/wes.',release,'.gene.cn'), 
sep = ',', quote = F, row.names = T)

## Validation step

In [ ]:
genecn = pd.read_csv('temp/wes.'+release+'.gene.cn', sep = ',')

In [ ]:
prevsegmentcn = pd.read_csv('temp/cnv_ccle.called.seg', sep = '\t')
segmentcn = pd.read_csv('temp/wes.'+release+'.segmented.cn', sep = ',')

In [ ]:
genecn.index = [i[-1] for i in genecn.index.str.split('_').tolist()]
segmentcn["DepMap_ID"] = [i[-1] for i in segmentcn["DepMap_ID"].str.split('_').tolist()]

In [ ]:
embargoed = "WES_embargoed.txt"
embargoed = [line.rstrip('\n') for line in open(embargoed)]
blacklisted = "blacklist.txt"
blacklisted = [line.rstrip('\n') for line in open(blacklisted)]

In [ ]:
prev = set(tc.get(name='segmented-cn-wes-prioritzed-7fe1', version=27, file='wes.19Q3interim.gene').index.values.tolist())
new1 = set(genecn.index.values.tolist())
new2 = set(segmentcn['Sample'].values.tolist())
print(len(prev), len(prev & new1), len(new1), len(new1 & new2))

In [ ]:
checkAmountOfSegments(segmentcn,thresh = 750)

In [ ]:
checkGeneChangeAccrossAll(genecn, thresh=1.5)

In [ ]:
newsamples =  list(set(segmentcn["Sample"].tolist()))

In [ ]:
segmentcn[segmentcn["DepMap_ID"]=="ACH-000658"]

In [ ]:
newsamples= refwm.get_sample_sets().loc[sample_set_id].samples

In [ ]:
samples = refwm.get_samples()
plots = samples.loc[samples.index.isin(newsamples)]["modeled_segments_plot_tumor"].values
for plot in plots:
    ! gsutil cp $plot temp/

In [ ]:
for plot in plots:
    display(Image('temp/'+plot.split('/')[-1]))

In [ ]:
genecn.to_csv('temp/wes.'+release+'.gene.cn')
segmentcn.to_csv('temp/wes.'+release+'.segmented.cn')

In [ ]:
## for genecn removing first blacklisted, then embargoed, to create two datasets
genecn = genecn.apply(lambda x: np.log2(1+x))
for i in genecn.index:
    if i in blacklisted:
        genecn = genecn.drop(i)
genecn.to_csv('temp/wes.'+release+'.blacklisted.gene.cn')
for i in genecn.index:
    if i in embargoed:
        genecn = genecn.drop(i)
genecn.to_csv('temp/wes.'+release+'.embargoed.gene.cn')

In [ ]:
## for segment removing first blacklisted, then embargoed, to create two datasets
segmentcn['Segment_Mean'] = segmentcn['Segment_Mean'].apply(lambda x: np.log2(1+x))
for i, val in segmentcn.iterrows():
    if val['DepMap_ID'] in blacklisted:
        segmentcn = segmentcn.drop(i)
segmentcn.to_csv('temp/wes.'+release+'.blacklisted.segmented.cn')
for i, val in segmentcn.iterrows():
    if val['DepMap_ID'] in embargoed:
        segmentcn = segmentcn.drop(i)
segmentcn.to_csv('temp/wes.'+release+'.embargoed.segmented.cn')

In [ ]:
segmentcn

# Upload to taiga

In [ ]:
tc.update_dataset(dataset_permaname="segmented-cn-wes-prioritzed-7fe1", 
                  upload_file_path_dict={'temp/wes.'+release+'.gene.cn': 'NumericMatrixCSV',
                                        'temp/wes.'+release+'.segmented.cn': 'TableCSV',
                                        'temp/wes.'+release+'.embargoed.gene.cn': 'NumericMatrixCSV','temp/wes.'+release+'.embargoed.segmented.cn': 'TableCSV',
                                         'temp/wes.'+release+'.blacklisted.gene.cn': 'NumericMatrixCSV',
                                         'temp/wes.'+release+'.blacklisted.segmented.cn': 'TableCSV'},
                dataset_description="updating to "+release+". Some cells lines have been flagged as: \n \
                - having bad looking copy ration plots = [] \n \
                 - having too many segments = [] \n \
                  - Genes having a similar CN value accross all []",
                 )